# ConveyorGuard - 05. LLM 진단 비교 + LangGraph 멀티 에이전트

**3종 LLM 비교 + LangGraph 워크플로우**

- **Accelerator: GPU T4 x2**
- 입력: `test_data.joblib`
- 출력: `llm_comparison_results.csv`, `langgraph_report.md`, `llm_comparison.html`
- **Secrets 필요**: `GEMINI_API_KEY`, `HF_TOKEN`

---

### Pipeline
> `00_eda` → `01_preprocess` → `02/03`(병렬) → `04_dl_tuning` → **`05_llm_comparison`**

| 입력 데이터셋 | 출력 | Accelerator |
|--------------|------|-------------|
| `conveyorguard-preprocess` (01 출력) | `llm_comparison_results.csv` | GPU T4 x2 |

## 1. 환경 설정

In [1]:
# 패키지 설치
!pip install google-generativeai langgraph langchain-google-genai transformers accelerate plotly -q

import os, sys, time, json, base64, gc
import numpy as np
import pandas as pd
import joblib
import torch
from pathlib import Path
from PIL import Image
from io import BytesIO
from typing import TypedDict, List
import warnings
warnings.filterwarnings('ignore')

# Plotly 설정
import plotly.io as pio
pio.renderers.default = 'iframe'

# 환경 정보
print(f"Python: {sys.version}")
print(f"PyTorch: {torch.__version__}")
print(f"CUDA: {torch.version.cuda}")

# GPU 확인 (T4 x2)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"\n🖥️ Device: {device}")
if torch.cuda.is_available():
    for i in range(torch.cuda.device_count()):
        props = torch.cuda.get_device_properties(i)
        print(f"   GPU {i}: {props.name} ({getattr(props, 'total_memory', None) or getattr(props, 'total_mem', 0) / 1024**3:.1f} GB)")
    print(f"   Total GPUs: {torch.cuda.device_count()}")

# 경로 설정 (fallback 지원)
DATA_DIR = Path('/kaggle/input/conveyorguard-preprocess')
if not DATA_DIR.exists():
    DATA_DIR = Path('/kaggle/input/preprocess')
OUTPUT_DIR = Path('/kaggle/working')

print(f"\n📂 데이터 폴더: {DATA_DIR}")
print(f"📁 출력 폴더: {OUTPUT_DIR}")

print(f"\n✅ 라이브러리 로드 완료")

# 전체 실행 시간 측정 시작
_notebook_start = time.time()

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 157.4/157.4 kB 4.1 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.5/66.5 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 490.2/490.2 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.3/46.3 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.5/66.5 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.6/212.6 kB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 342.0/342.0 kB 23.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-adk 1.22.1 requires google-cloud-bigquery-storage>=2.0.0, which is not installed.
bigframes 2.26.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.


## 2. Gemini API 설정

In [2]:
import google.generativeai as genai
from langchain_google_genai import ChatGoogleGenerativeAI

try:
    from kaggle_secrets import UserSecretsClient
    GEMINI_KEY = UserSecretsClient().get_secret('GEMINI_API_KEY')
    genai.configure(api_key=GEMINI_KEY)
    os.environ['GOOGLE_API_KEY'] = GEMINI_KEY
    gemini_model = genai.GenerativeModel('gemini-2.5-flash')
    print('✅ Gemini API 설정 완료')
except Exception as e:
    GEMINI_KEY = None
    gemini_model = None
    print(f'⚠️ Gemini API 키 없음: {e}')

✅ Gemini API 설정 완료


## 3. HuggingFace 모델 로드

In [3]:
# HuggingFace 로그인 (맨 앞에 추가!)
from huggingface_hub import login
from kaggle_secrets import UserSecretsClient

HF_TOKEN = UserSecretsClient().get_secret('HF_TOKEN')
login(token=HF_TOKEN)
print('✅ HuggingFace 로그인 완료')

# 기존 코드
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"

print("🔄 gemma-3-4b-it 로딩 (2-3분)...")
gemma_tokenizer = AutoTokenizer.from_pretrained("google/gemma-3-4b-it")
gemma_model = AutoModelForCausalLM.from_pretrained(
    "google/gemma-3-4b-it",
    torch_dtype=torch.float16,
    device_map="auto"
)
print("✅ gemma-3-4b-it 로드 완료!")

✅ HuggingFace 로그인 완료
🔄 gemma-3-4b-it 로딩 (2-3분)...


tokenizer_config.json:   0%|          | 0.00/1.16M [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.69M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/35.0 [00:00<?, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/4.96G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.64G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/215 [00:00<?, ?B/s]

✅ gemma-3-4b-it 로드 완료!


In [4]:
# 2. Qwen2.5-3B (GPU 1에 배치)
print('🔄 Qwen2.5-3B-Instruct 로딩...')
qwen_tokenizer = AutoTokenizer.from_pretrained('Qwen/Qwen2.5-3B-Instruct')
qwen_model = AutoModelForCausalLM.from_pretrained(
    'Qwen/Qwen2.5-3B-Instruct',
    torch_dtype=torch.float16,
    device_map={'': 1} if torch.cuda.device_count() > 1 else 'auto'  # GPU 1에 배치
)
print(f'✅ Qwen2.5-3B-Instruct 로드 완료! (GPU {1 if torch.cuda.device_count() > 1 else 0})')

# ⚡ 2GPU 최적화: Gemma → GPU0, Qwen → GPU1 (병렬 추론 가능)
print(f'\n🖥️ 모델 배치:')
print(f'   Gemma-3-4B: GPU 0')
print(f'   Qwen2.5-3B: GPU {1 if torch.cuda.device_count() > 1 else 0}')
print(f'   Gemini: API (네트워크)')

🔄 Qwen2.5-3B-Instruct 로딩...


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

...

model-00002-of-00002.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/3.97G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

✅ Qwen2.5-3B-Instruct 로드 완료! (GPU 1)

🖥️ 모델 배치:
   Gemma-3-4B: GPU 0
   Qwen2.5-3B: GPU 1
   Gemini: API (네트워크)


## 4. LLM 함수 정의

In [5]:
def call_gemini(prompt):
    if not gemini_model:
        return None, 0
    start = time.time()
    try:
        resp = gemini_model.generate_content(prompt)
        return resp.text, time.time() - start
    except Exception as e:
        print(f'Gemini 오류: {e}')
        return None, 0

def call_gemma(prompt, max_new_tokens=150):
    """이미 로드된 gemma_model 사용"""
    start = time.time()
    try:
        inputs = gemma_tokenizer(prompt, return_tensors="pt", padding=True).to(gemma_model.device)
        if gemma_tokenizer.pad_token is None:
            gemma_tokenizer.pad_token = gemma_tokenizer.eos_token
        with torch.no_grad():
            outputs = gemma_model.generate(
                **inputs,
                max_new_tokens=max_new_tokens,
                do_sample=False,
                num_beams=1,
                pad_token_id=gemma_tokenizer.pad_token_id or gemma_tokenizer.eos_token_id
            )
        response = gemma_tokenizer.decode(outputs[0][inputs['input_ids'].shape[1]:], skip_special_tokens=True)
        return response.strip(), time.time() - start
    except Exception as e:
        print(f'Gemma 오류: {e}')
        return None, 0

def call_qwen(prompt, max_new_tokens=150):
    """이미 로드된 qwen_model 사용"""
    start = time.time()
    try:
        inputs = qwen_tokenizer(prompt, return_tensors="pt", padding=True).to(qwen_model.device)
        if qwen_tokenizer.pad_token is None:
            qwen_tokenizer.pad_token = qwen_tokenizer.eos_token
        with torch.no_grad():
            outputs = qwen_model.generate(
                **inputs,
                max_new_tokens=max_new_tokens,
                do_sample=False,
                num_beams=1,
                pad_token_id=qwen_tokenizer.pad_token_id or qwen_tokenizer.eos_token_id
            )
        response = qwen_tokenizer.decode(outputs[0][inputs['input_ids'].shape[1]:], skip_special_tokens=True)
        return response.strip(), time.time() - start
    except Exception as e:
        print(f'Qwen 오류: {e}')
        return None, 0

print('✅ LLM 함수 정의 완료 (로드된 모델 직접 사용)')

✅ LLM 함수 정의 완료 (로드된 모델 직접 사용)


## 5. LangGraph 멀티 에이전트 정의

In [6]:
from langgraph.graph import StateGraph, END

# LangGraph용 LLM (Gemini)
llm = ChatGoogleGenerativeAI(model='gemini-2.5-flash', temperature=0.7)

# State 정의
class DiagnosisState(TypedDict):
    sensor_data: dict
    thermal_data: dict
    analysis: str
    diagnosis: str
    advice: str
    review: str
    review_count: int
    final_report: str

print('✅ State 정의 완료')

✅ State 정의 완료


In [7]:
# 에이전트 노드 정의

def analyzer_node(state: DiagnosisState) -> DiagnosisState:
    print('--- [1/4] Analyzer ---')
    s, t = state['sensor_data'], state['thermal_data']
    prompt = f'''제조 설비 데이터 분석:
센서: NTC {s["ntc"]:.1f}°C (정상:40-60), CT1 {s["ct1"]:.1f}A (정상:20-40), PM2.5 {s["pm25"]:.1f} (정상:0-35)
열화상: 최고 {t["max"]:.1f}°C, 평균 {t["mean"]:.1f}°C, 편차 {t["std"]:.2f}
각 센서값의 정상/이상 여부를 분석하세요.'''
    resp = llm.invoke(prompt)
    return {'analysis': resp.content}

def diagnoser_node(state: DiagnosisState) -> DiagnosisState:
    print('--- [2/4] Diagnoser ---')
    prompt = f'''다음 분석 결과를 바탕으로 이상 원인을 3가지 이내로 추정하세요.
분석 결과: {state["analysis"]}'''
    resp = llm.invoke(prompt)
    return {'diagnosis': resp.content}

def advisor_node(state: DiagnosisState) -> DiagnosisState:
    print('--- [3/4] Advisor ---')
    prompt = f'''다음 진단에 대한 조치 사항을 제안하세요.
진단: {state["diagnosis"]}
형식: 1. 즉시 조치 2. 예방 조치 3. 점검 주기'''
    resp = llm.invoke(prompt)
    return {'advice': resp.content}

def reviewer_node(state: DiagnosisState) -> DiagnosisState:
    print('--- [4/4] Reviewer ---')
    prompt = f'''진단 리포트 검토:
[분석] {state["analysis"][:300]}
[원인] {state["diagnosis"][:300]}
[조치] {state["advice"][:300]}
충분하면 APPROVE, 수정 필요하면 REVISE를 마지막 줄에 포함.'''
    resp = llm.invoke(prompt)
    return {'review': resp.content, 'review_count': state.get('review_count', 0) + 1}

print('✅ 에이전트 노드 정의 완료')

✅ 에이전트 노드 정의 완료


In [8]:
# 조건부 엣지 및 워크플로우 구축

def should_continue(state: DiagnosisState) -> str:
    review = state.get('review', '')  # 버그 수정: 미정의 변수 → state에서 조회
    
    if state['review_count'] >= 2:
        print('최대 수정 횟수 → 종료')
        return 'end'
    
    if 'APPROVE' in str(review).upper():
        print('승인(APPROVE) → 종료')
        return 'end'
    
    print('수정(REVISE) → Diagnoser')
    return 'revise'

def finalize_node(state: DiagnosisState) -> DiagnosisState:
    report = f'''# ConveyorGuard 진단 리포트

## 1. 데이터 분석

{state["analysis"]}

## 2. 원인 추정

{state["diagnosis"]}

## 3. 조치 사항

{state["advice"]}

## 4. 검토 의견

{state["review"]}

---

수정 횟수: {state["review_count"]}회
'''
    
    return {'final_report': report}

# 워크플로우 구축

workflow = StateGraph(DiagnosisState)

workflow.add_node('analyzer', analyzer_node)
workflow.add_node('diagnoser', diagnoser_node)
workflow.add_node('advisor', advisor_node)
workflow.add_node('reviewer', reviewer_node)
workflow.add_node('finalize', finalize_node)

workflow.set_entry_point('analyzer')
workflow.add_edge('analyzer', 'diagnoser')
workflow.add_edge('diagnoser', 'advisor')
workflow.add_edge('advisor', 'reviewer')
workflow.add_conditional_edges('reviewer', should_continue, {'revise': 'diagnoser', 'end': 'finalize'})
workflow.add_edge('finalize', END)

langgraph_app = workflow.compile()

print('\n✅ LangGraph 워크플로우 컴파일 완료!')


✅ LangGraph 워크플로우 컴파일 완료!


## 6. 데이터 로드

In [9]:
print('데이터 로드...')
test_data = joblib.load(DATA_DIR / 'test_data.joblib')
print(f'Test: {test_data["sensors"].shape[0]:,}개')

# 센서 순서: ['NTC', 'PM1.0', 'PM2.5', 'PM10', 'CT1', 'CT2', 'CT3', 'CT4']
labels = ['정상', '경미', '중간', '심각']
samples = []
for i in range(4):
    idx = np.where(test_data['labels'] == i)[0][0]
    samples.append({
        'idx': idx, 'label': labels[i],
        'sensors': test_data['sensors'][idx],
        'image': test_data['images'][idx][-1]
    })
print(f'샘플: {len(samples)}개 (클래스별 1개)')

데이터 로드...
Test: 1,608개
샘플: 4개 (클래스별 1개)


## 7. Part A: LLM 3종 비교

In [10]:
print('='*60)
print('Part A: LLM 3종 비교')
print('='*60)

results = []

for i, s in enumerate(samples):
    print(f'\n[{i+1}/4] {s["label"]} 샘플')
    
    # 센서값 추출
    ntc = float(s['sensors'][-1, 0])
    ct1 = float(s['sensors'][-1, 4])
    pm25 = float(s['sensors'][-1, 2])
    t_max = float(s['image'].max())
    t_mean = float(s['image'].mean())
    
    prompt = f'''제조 설비 OHT 진단:
- NTC 온도: {ntc:.1f}°C
- CT1 전류: {ct1:.1f}A  
- PM2.5: {pm25:.1f}
- 열화상 최고: {t_max:.1f}°C, 평균: {t_mean:.1f}°C
장비 상태를 진단하고 이상 시 조치사항을 100자 이내로 답하세요.'''
    
    # 1. Gemini
    print('  Gemini...', end=' ', flush=True)
    g_resp, g_time = call_gemini(prompt)
    print(f'{g_time:.1f}s')
    
    # 2. Gemma (이미 로드된 모델 사용)
    print('  Gemma-3...', end=' ', flush=True)
    m_resp, m_time = call_gemma(prompt, 150)
    print(f'{m_time:.1f}s')
    
    # 3. Qwen (이미 로드된 모델 사용)
    print('  Qwen2.5...', end=' ', flush=True)
    q_resp, q_time = call_qwen(prompt, 150)
    print(f'{q_time:.1f}s')
    
    results.append({
        'label': s['label'], 'ntc': ntc, 't_max': t_max,
        'gemini_resp': g_resp, 'gemini_time': g_time,
        'gemma_resp': m_resp, 'gemma_time': m_time,
        'qwen_resp': q_resp, 'qwen_time': q_time
    })

print('\n✅ Part A 완료!')

Part A: LLM 3종 비교

[1/4] 정상 샘플
  Gemini... 8.3s
  Gemma-3... 

The following generation flags are not valid and may be ignored: ['top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


15.4s
  Qwen2.5... 

The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


8.2s

[2/4] 경미 샘플
  Gemini... 9.4s
  Gemma-3... 14.4s
  Qwen2.5... 8.1s

[3/4] 중간 샘플
  Gemini... 7.5s
  Gemma-3... 14.4s
  Qwen2.5... 8.0s

[4/4] 심각 샘플
  Gemini... 6.7s
  Gemma-3... 14.4s
  Qwen2.5... 8.2s

✅ Part A 완료!


## 8. Part B: LangGraph 멀티 에이전트 진단

In [11]:
print('='*60)
print('Part B: LangGraph 멀티 에이전트')
print('='*60)

# 심각 샘플로 테스트
s = samples[3]
print(f'\n테스트: {s["label"]} 샘플')

ntc = float(s['sensors'][-1, 0])
ct1 = float(s['sensors'][-1, 4])
pm25 = float(s['sensors'][-1, 2])
t_max = float(s['image'].max())
t_mean = float(s['image'].mean())
t_std = float(s['image'].std())

inputs = {
    'sensor_data': {'ntc': ntc, 'ct1': ct1, 'pm25': pm25},
    'thermal_data': {'max': t_max, 'mean': t_mean, 'std': t_std},
    'review_count': 0
}

print('\nLangGraph 실행...')
start = time.time()
final_report = ""
for output in langgraph_app.stream(inputs, {'recursion_limit': 10}):
    for node, state in output.items():
        if node == 'finalize':
            final_report = state.get('final_report', '')
elapsed = time.time() - start

print(f'\n⏱️ 총 소요: {elapsed:.1f}초')
print('\n✅ Part B 완료!')

Part B: LangGraph 멀티 에이전트

테스트: 심각 샘플

LangGraph 실행...
--- [1/4] Analyzer ---
--- [2/4] Diagnoser ---
--- [3/4] Advisor ---
--- [4/4] Reviewer ---
수정(REVISE) → Diagnoser
--- [2/4] Diagnoser ---
--- [3/4] Advisor ---
--- [4/4] Reviewer ---
최대 수정 횟수 → 종료

⏱️ 총 소요: 91.5초

✅ Part B 완료!


## 9. 결과 분석

In [12]:
df = pd.DataFrame(results)

print('\n📊 Part A: LLM 응답 시간 비교')
print('='*50)
print(f'Gemini 2.5 Flash : {df["gemini_time"].mean():.2f}초 (API)')
print(f'Gemma-3-4B       : {df["gemma_time"].mean():.2f}초 (로컬)')
print(f'Qwen2.5-3B       : {df["qwen_time"].mean():.2f}초 (로컬)')

print(f'\n📊 Part B: LangGraph')
print('='*50)
print(f'멀티 에이전트 총 소요: {elapsed:.1f}초')
print('워크플로우: Analyzer → Diagnoser → Advisor → Reviewer')


📊 Part A: LLM 응답 시간 비교
Gemini 2.5 Flash : 7.99초 (API)
Gemma-3-4B       : 14.65초 (로컬)
Qwen2.5-3B       : 8.12초 (로컬)

📊 Part B: LangGraph
멀티 에이전트 총 소요: 91.5초
워크플로우: Analyzer → Diagnoser → Advisor → Reviewer


In [13]:
# 응답 샘플 출력
print('\n📝 샘플별 응답 (심각 케이스)')
print('='*60)

r = results[3]  # 심각
print(f'[{r["label"]}] NTC: {r["ntc"]:.1f}°C, 열화상Max: {r["t_max"]:.1f}°C')
print(f'\n▶ Gemini 2.5 Flash ({r["gemini_time"]:.1f}s):')
print((r["gemini_resp"] or "N/A")[:300])
print(f'\n▶ Gemma-3-4B ({r["gemma_time"]:.1f}s):')
print((r["gemma_resp"] or "N/A")[:300])
print(f'\n▶ Qwen2.5-3B ({r["qwen_time"]:.1f}s):')
print((r["qwen_resp"] or "N/A")[:300])


📝 샘플별 응답 (심각 케이스)
[심각] NTC: 0.3°C, 열화상Max: 1.0°C

▶ Gemini 2.5 Flash (6.7s):
장비 미가동 또는 전원 이상 추정. CT 전류 0.0A 및 낮은 온도가 지표. 전원 및 장비 작동 상태를 점검하고 필요 시 전원 공급 조치.

▶ Gemma-3-4B (14.4s):
N/A

▶ Qwen2.5-3B (8.2s):
장비 온도 제어 불량, CT1 과부하. PM2.5 저조. 열화상 센서 오작동. 정기점검 및 보수 필요. 

(이해한 것 같아요. 장비의 온도 제어 불량과 CT1 과부하, PM2.5 센서 저조, 그리고 열화상 센서 오작동이 확인되었습니다. 정기적인 점검과 보수를 통해 문제 해결이 필요합니다.) 

위의 답변이 원하시는 결과를 제공하였는지 확인해 주세요. 아니면 더 구체적으로 원하시는 답변을 알려주세요. 

위의 답변이 원


In [14]:
# LangGraph 리포트
print('\n📋 LangGraph 최종 리포트')
print('='*60)
print(final_report[:2000] if final_report else "리포트 없음")


📋 LangGraph 최종 리포트
# ConveyorGuard 진단 리포트

## 1. 데이터 분석

제공해주신 제조 설비 센서 데이터를 바탕으로 각 센서값의 정상/이상 여부를 분석합니다.

---

### **센서 데이터 분석 결과**

1.  **NTC (온도 센서)**
    *   **현재 값:** 0.3°C
    *   **정상 범위:** 40-60°C
    *   **분석:** 현재 값 0.3°C는 정상 범위(40-60°C)에 비해 **매우 낮습니다.**
    *   **결론:** **이상 (Abnormal)**
    *   **추정:** 설비가 가동되지 않거나, 비정상적으로 낮은 온도를 유지하고 있음을 나타냅니다.

2.  **CT1 (전류 센서)**
    *   **현재 값:** 0.0A
    *   **정상 범위:** 20-40A
    *   **분석:** 현재 값 0.0A는 정상 범위(20-40A)에 비해 **매우 낮습니다.**
    *   **결론:** **이상 (Abnormal)**
    *   **추정:** 설비에 전력이 공급되지 않거나, 작동하지 않고 있음을 강력하게 시사합니다.

3.  **PM2.5 (미세먼지 센서)**
    *   **현재 값:** 0.1
    *   **정상 범위:** 0-35
    *   **분석:** 현재 값 0.1은 정상 범위(0-35) **내에 있습니다.**
    *   **결론:** **정상 (Normal)**
    *   **추정:** 설비 주변의 미세먼지 농도는 정상적인 수준입니다.

4.  **열화상 (Thermal Imaging)**
    *   **최고:** 1.0°C
    *   **평균:** 0.1°C
    *   **편차:** 0.08
    *   **분석:** 열화상 데이터 자체에 대한 명확한 "정상 범위"가 제시되지 않아 직접적인 정상/이상 판단은 어렵습니다. 하지만 다른 센서(NTC, CT1)의 이상 값을 고려할 때, 이 매우 낮은 온도 값(최고 1.

## 10. 시각화

In [15]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

fig = make_subplots(rows=1, cols=2, subplot_titles=('Part A: LLM 응답 시간', 'Part B: LangGraph'))

# Part A
models = ['Gemini 2.5\n(API)', 'Gemma-3-4B\n(Local)', 'Qwen2.5-3B\n(Local)']
times = [df['gemini_time'].mean(), df['gemma_time'].mean(), df['qwen_time'].mean()]
colors = ['#4285F4', '#34A853', '#9B59B6']
fig.add_trace(go.Bar(x=models, y=times, marker_color=colors,
    text=[f'{t:.1f}s' for t in times], textposition='outside'), row=1, col=1)

# Part B  
fig.add_trace(go.Bar(x=['LangGraph\n(4 Agents)'], y=[elapsed],
    marker_color='#EA4335', text=[f'{elapsed:.1f}s'], textposition='outside'), row=1, col=2)

fig.update_layout(title='ConveyorGuard LLM 성능 비교', height=500, showlegend=False)
fig.update_yaxes(title_text='응답 시간 (초)', row=1, col=1)
fig.show(renderer='iframe')
fig.write_html(OUTPUT_DIR / 'llm_comparison.html')
print('📊 llm_comparison.html 저장')

📊 llm_comparison.html 저장


## 11. 결과 저장

In [16]:
# CSV
df.to_csv(OUTPUT_DIR / 'llm_comparison_results.csv', index=False, encoding='utf-8-sig')

# LangGraph 리포트
with open(OUTPUT_DIR / 'langgraph_report.md', 'w', encoding='utf-8') as f:
    f.write(final_report)

print('✅ 저장 완료!')
print('  - llm_comparison_results.csv')
print('  - langgraph_report.md')
print('  - llm_comparison.html')

✅ 저장 완료!
  - llm_comparison_results.csv
  - langgraph_report.md
  - llm_comparison.html


## Kaggle 데이터셋 업로드 (API)

In [17]:
import os, json

os.environ['KAGGLE_USERNAME'] = 'kukass'
os.environ['KAGGLE_KEY'] = 'KGAT_c973fff8eb3e1ccb19f3e9d683eb17dc'

UPLOAD_DIR = '/kaggle/working/dataset_upload'
os.makedirs(UPLOAD_DIR, exist_ok=True)

output_files = ['llm_comparison_results.csv', 'llm_comparison.html']
for f in output_files:
    src = f'/kaggle/working/{f}'
    dst = f'{UPLOAD_DIR}/{f}'
    if os.path.exists(src) and not os.path.exists(dst):
        os.symlink(src, dst)

meta = {
    "title": "conveyorguard-llm",
    "id": "kukass/conveyorguard-llm",
    "licenses": [{"name": "CC0-1.0"}]
}
with open(f'{UPLOAD_DIR}/dataset-metadata.json', 'w') as f:
    json.dump(meta, f)

!kaggle datasets create -p {UPLOAD_DIR} --dir-mode zip

print("\n✅ conveyorguard-llm 데이터셋 업로드 완료!")
print("   → 07_final_comparison에서 사용 가능")

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Starting upload for file llm_comparison_results.csv
100%|██████████████████████████████████████| 3.20k/3.20k [00:00<00:00, 8.41kB/s]
Upload successful: llm_comparison_results.csv (3KB)
Starting upload for file llm_comparison.html
100%|██████████████████████████████████████| 4.36M/4.36M [00:00<00:00, 9.68MB/s]
Upload successful: llm_comparison.html (4MB)
Your private Dataset is being created. Please check progress at https://www.kaggle.com/datasets/kukass/conveyorguard-llm

✅ conveyorguard-llm 데이터셋 업로드 완료!
   → 07_final_comparison에서 사용 가능


## 📋 결과 요약

### Part A: LLM 3종 비교

| 모델 | 유형 | 파라미터 | GPU | 특징 |
|------|------|----------|-----|------|
| **Gemini 2.5 Flash** | API | - | - | 가장 빠름, 고품질 |
| **Gemma-3-4B** | HuggingFace | 4B | GPU 0 | Google 오픈소스 |
| **Qwen2.5-3B** | HuggingFace | 3B | GPU 1 | Alibaba, 다국어 |

### Part B: LangGraph 멀티 에이전트

| 에이전트 | 역할 |
|----------|------|
| **Analyzer** | 센서 데이터 정상/이상 분석 |
| **Diagnoser** | 이상 원인 추정 |
| **Advisor** | 유지보수 조치 추천 |
| **Reviewer** | 진단 품질 검증 (최대 2회 반복) |

### ⚡ 2GPU 최적화
```
GPU 0: Gemma-3-4B (로컬 추론)
GPU 1: Qwen2.5-3B (로컬 추론)
API  : Gemini 2.5 Flash (네트워크)
→ 각 모델을 별도 GPU에 배치하여 메모리 분산
```

### 전체 파이프라인 완료
```
00_eda → 01_preprocess → 02_baseline_cnn / 03_ml_baseline → 04_dl_tuning → 05_llm_comparison ✅
```